In [ ]:
# !ln -s ~/Cityscapes/ ./Cityscapes

In [ ]:
# from torchvision import datasets
# # https://pytorch.org/vision/0.8/datasets.html#cityscapes
# # split (string, optional) – The image split to use, train, test or val if mode=”fine” otherwise train, train_extra or val
# # dataset = datasets.Cityscapes('./data/cityscapes', split='train', mode='fine', target_type='semantic')
# target_type = ['instance', 'semantic', 'polygon', 'color']
# dataset = datasets.Cityscapes('./Cityscapes', split='val', mode='fine', target_type=target_type)

In [4]:
import torch
import torchvision
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.datasets import CocoDetection

import utils
from coco_utils import get_city
import transforms

# Load a model pre-trained on COCO and put it in inference mode

print('Loading pretrained model...')
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Load the COCO 2017 train and val sets. We use the CocoDetection class definition
# from ./coco_utils.py, not the original torchvision.CocoDetection class. Also, we
# use transforms from ./transforms, not torchvision.transforms, because they need
# to transform the bboxes and masks along with the image.

# coco_path = "./COCO"

preprocess = transforms.Compose([
    transforms.ToTensor()
])
# print('Loading COCO train, val datasets...')
coco_val_dataset = get_city(preprocess)
# coco_val_dataset = get_coco(coco_path, 'val', transform)

def collate_fn(batch):
    return tuple(zip(*batch))

val_dataloader = torch.utils.data.DataLoader(coco_val_dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn=collate_fn)

Loading pretrained model...
loading annotations into memory...
Done (t=0.63s)
creating index...
index created!


In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [5]:
images, targets = next(iter(val_dataloader))

# print(type(images), len(images))
# print(images[0].shape)
# print(type(targets), len(targets))
# print(targets[0].keys())
# # print()
# # images = images.to(device)

images = [ img.cuda() for img in images ]
images.to(device)
predictions = model(images)

print('Prediction keys:', list(dict(predictions[0])))
print('Boxes shape:', predictions[0]['boxes'].shape)
print('Labels shape:', predictions[0]['labels'].shape)
print('Scores shape:', predictions[0]['scores'].shape)
print('Masks shape:', predictions[0]['masks'].shape)

In [ ]:
images, targets = next(iter(val_dataloader))

print(type(images), len(images))
print(images[0].shape)
print(type(targets), len(targets))
print(targets[0][2])

# from matplotlib import pyplot as plt

# plt.figure(1, figsize=(12, 9), dpi=100)
# plt.imshow(targets[0][0])
# plt.title('Validation image result')
# plt.show()

# images = images.to(device)

# images = [ img.to(device) for img in images ]
# predictions = model(images)

# print('Prediction keys:', list(dict(predictions[0])))
# print('Boxes shape:', predictions[0]['boxes'].shape)
# print('Labels shape:', predictions[0]['labels'].shape)
# print('Scores shape:', predictions[0]['scores'].shape)
# print('Masks shape:', predictions[0]['masks'].shape)